In [20]:
#importing the libraries
#couple of things to note - a short strangle is not optimal for a portfolio given the following requirements:
#1) need to be okay with a naked/covered option
#2) expect the price to swing wildly
#3) need to know the stock loan fee - differs according to the platform used

import pandas as pd
import yfinance as yf
import numpy as np

In [21]:
port_pick = pd.read_csv(r'portfolio_pick.csv')['Stock'].tolist()

In [22]:
def calculate_avg_sd(ticker, start_date, end_date):
    try:
        data = yf.download(ticker, start=start_date, end=end_date)
        if data.empty:
            raise ValueError(f"No data returned for ticker {ticker}")
        avg_price = data['Close'].mean()
        sd_price = data['Close'].std()
        return avg_price, sd_price
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None, None

In [23]:
def fetch_options_data(tickers):
    options_dict = {}
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            options = []
            if not stock.options:
                raise ValueError(f"No options data available for ticker {ticker}")
            for expiry in stock.options:
                opt_chain = stock.option_chain(expiry)
                calls = opt_chain.calls
                puts = opt_chain.puts

                calls['Type'] = 'Call'
                puts['Type'] = 'Put'
                calls['Expiration'] = expiry
                puts['Expiration'] = expiry

                options.append(pd.concat([calls, puts]))

            options_dict[ticker] = pd.concat(options, ignore_index=True)
        except Exception as e:
            print(f"Error fetching options for {ticker}: {e}")
            options_dict[ticker] = pd.DataFrame()
    return options_dict

In [24]:
def filter_by_sd(options_dict, avg_sd_dict):
    sd1 = []
    sd2 = []

    for ticker, options_df in options_dict.items():
        if options_df.empty:
            continue

        avg_price, sd_price = avg_sd_dict.get(ticker, (None, None))
        if avg_price is None or sd_price is None:
            continue

        within_sd1 = options_df[(options_df['strike'] >= avg_price - sd_price) &
                                (options_df['strike'] <= avg_price + sd_price)]
        within_sd2 = options_df[(options_df['strike'] >= avg_price - 2 * sd_price) &
                                (options_df['strike'] <= avg_price + 2 * sd_price)]

        within_sd2 = within_sd2[~within_sd2.index.isin(within_sd1.index)]

        sd1.append(within_sd1)
        sd2.append(within_sd2)

    return pd.concat(sd1, ignore_index=True), pd.concat(sd2, ignore_index=True)

In [25]:
if __name__ == "__main__":
    from datetime import datetime, timedelta

    end_date = datetime.today()
    start_date = end_date - timedelta(days=30)

    if not isinstance(port_pick, list):
        raise TypeError("'port_pick' must be a list of tickers.")
    if not all(isinstance(ticker, str) for ticker in port_pick):
        raise ValueError("All elements in 'port_pick' must be strings representing tickers.")

    avg_sd_dict = {
        ticker: calculate_avg_sd(ticker, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
        for ticker in port_pick
    }

    options_dict = fetch_options_data(port_pick)

    sd1_df, sd2_df = filter_by_sd(options_dict, avg_sd_dict)

#run tiem 48.9 seconds

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******